In [16]:
import spacy
from spacy.vectors import Vectors
import os
import pandas as pd
import numpy as np
nlp = spacy.load('en_core_web_sm')
vectors = Vectors(shape=(10000, 300))
nlp.vocab.vectors = vectors
print(nlp.vocab.vectors.shape)
import re
re_c = re.compile(r'\w+')
import warnings
warnings.filterwarnings('ignore')

(10000, 300)


In [2]:
# switch for debug
flag_print = True

# switch to clear existing data
flag_clear = True

#threshold value for determining section
threshold = 0.5

In [3]:
similar_to = {
    'edu' : ['education', 'study', 'academics', 'institute', 'school', 'college'],
    'exp' : ['job', 'internship', 'training', 'research', 'career', 'profession', 'role'
             'project', 'responsibility', 'description', 'work experience', 'workshop', 'conference'],
    'skill' : ['skill', 'languages', 'technology', 'framework', 'tools', 'database'],
    'extra' : ['introduction', 'intro', 'achievement', 'hobby', 'links', 'additional', 
               'personal', 'award', 'objective', 'miscellaneous', 'interest']
}

list_of_sections = similar_to.keys()

# to bring similar_words to their normal forms
for section in list_of_sections:
    new_list = []
    
    for word in similar_to[section]:
        docx = nlp(word)
        new_list.append(docx[0].lemma_)
        
    if flag_print:
        print(section, new_list)
        
    similar_to[section] = new_list

edu ['education', 'study', 'academics', 'institute', 'school', 'college']
exp ['job', 'internship', 'training', 'research', 'career', 'profession', 'roleproject', 'responsibility', 'description', 'work', 'workshop', 'conference']
skill ['skill', 'language', 'technology', 'framework', 'tool', 'database']
extra ['introduction', 'intro', 'achievement', 'hobby', 'link', 'additional', 'personal', 'award', 'objective', 'miscellaneous', 'interest']


In [4]:
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(mod_word)

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.
    
if flag_print:
    test_words = ['Hello!!', '.,<>', 'India', 'of', '..freedoM..', 'e-mail']
    
    for word in test_words:
        print(word, '--returned-->', modify(word))

Hello!! --returned--> hello
.,<> --returned--> None
India --returned--> india
of --returned--> None
..freedoM.. --returned--> freedom
e-mail --returned--> email


In [7]:
def is_empty(line):
    for c in line:
        if (c.isalpha()):
            return False
    return True
      
if flag_print:
    test_words = ['.', '<.>', 'Speak', 'out', '"Eric"', 'freemail...']
    


In [17]:
dict_of_data_series = {}
flag_print = False
path_cv = "/home/ubuntu/Documents/python works/unicept/b/Resume_Filtering-develop/Data/CVs/"
for file_name in os.listdir(path_cv):
    if flag_print:
        print('\n')
        print('*'*25) 
        print(file_name) 
        print('*'*25) 
        
    main_file_handler = open(path_cv+file_name, 'r', encoding='latin-1')  
    previous_section  = 'extra'
    
    curr_data_series = pd.Series([""]*len(list_of_sections), index=list_of_sections)
                   
    for line in main_file_handler:
        # skip line if empty
        if (len(line.strip()) == 0 or is_empty(line)):
            continue
                
        # processing next line
        list_of_words_in_line = re_c.findall(line)
        list_of_imp_words_in_line  = []
        
        for i in range(len(list_of_words_in_line)):
            modified_word = modify(list_of_words_in_line[i])
            
            if (modified_word):
                list_of_imp_words_in_line.append(modified_word)

        curr_line = ' '.join(list_of_imp_words_in_line)
        doc = nlp(curr_line)
        section_value = {}
            
       
        for section in list_of_sections:
            section_value[section] = 0.0
        section_value[None] = 0.0
            
         
        for token in doc:
            for section in list_of_sections:
                for word in similar_to[section]:
                    word_token = doc.vocab[word]
                    section_value[section] = max(section_value[section], float(word_token.similarity(token)))

        
        most_likely_section = None
        for section in list_of_sections:
            
            if (section_value[most_likely_section] < section_value[section] and section_value[section] > threshold):
                most_likely_section = section
            
 
        if (previous_section != most_likely_section and most_likely_section is not None):
            previous_section = most_likely_section
                

       
        try:
            docx = nlp(line)
        except:
            continue  # to handle the odd case of characters like 'x02', etc.
        mod_line = ''
        for token in docx:
            if (not token.is_stop):
                mod_line += token.lemma_ + ' '
        
        curr_data_series[previous_section] += mod_line
            
    dict_of_data_series[file_name] = curr_data_series
    if flag_print:
        print(curr_data_series)
    main_file_handler.close()
     
data_frame = pd.DataFrame(dict_of_data_series)
data_frame.to_csv('prc_data.csv', sep='\t')


In [18]:
data_frame.head()

,cv154,cv185,cv172,cv49,cv102,cv178,cv10,cv149,cv165,cv222,...,cv30,cv14,cv121,cv238,cv5,cv125,cv20,cv122,cv174,cv54
edu,education \n B.tech Electronics \n Sikshya o a...,education \n MCA Computer Application \n Pondi...,education \n shine Star Sr . Sec . school Jala...,project : school application \n school app mob...,education \n B.E. Engineering \n Bengal Engine...,,education \n,education \n b.tech ECE \n Vardhaman College E...,website store retrieve information Alumnis Col...,education \n mca \n TIT & science Bhopal Bhopa...,...,education \n b.tech computer science engineeri...,,education \n Bachelor Visvodaya Engineering \n...,,education \n b - Tech Computer Science & Engin...,education \n Computer Engineering \n Brahma Va...,education \n be(computer ) Computer Science \n...,,project : eLearning & education \n environment...,Indian Institute Technology Delhi . \n homepag...
exp,look professional career innovative skilled po...,work experience \n Java Developer End Develope...,work experience \n End developer \n Nugen Comp...,"â¢ work Software developer Sonora software , ...",* currently work Cognizant Technology Solution...,work experience \n - end developer \n OG SOFTW...,work experience \n Team Member(software develo...,motivate good attitude develop new skill use e...,build successful career field Information Tech...,work experience \n Java Developer \n chaitanya...,...,"growth orient challenge career , contribute kn...",secure position efficiently contribute skill a...,"pursue successful , challenging exciting caree...",â¢ experience work Relational Database ( RDBM...,work esteemed organization & secure challengin...,work experience \n Android Developer \n Applic...,work experience \n software developer \n Max S...,â working experience open source framework S...,work experience \n Sr End UI Developer \n Newt...,description : \n work pr \n imarily ...
skill,"LANGUAGES core JAVA , manual testing , SQL . \...","skill \n Java ( 1 year ) , Javascript ( 1 year...",use knowledge skill environment promote learni...,â¢ ability learn new application technology q...,Application Developer - Cognizant Technology S...,technology : html css \n role : - end develope...,information technology \n Nagpur university Na...,"skill \n Apache ( 1 year ) , API ( 1 year ) , ...","analytical , problem solve , communication int...",utilize educational skill improve company oper...,...,skill organization enhance experience continuo...,"skill \n AngularJs . , Mvc , ASP .net , css , ...",Hybrid Mobile Application developer good exper...,â¢ 7 + year experience prove expertise system...,skill \n Dot net ( 1 year ),"skill \n Php ( 1 year ) , Android Application ...","skill \n C ( 1 year ) , C++ ( 2 year ) , MFC (...","â comprehensive experience design , develop ...","Clarion Technologies - Pune , Maharashtra \n J...","information Retrieval , database Impleme..."
extra,Prasannera Das \n Banglore \n,"Arun Sundarapandian \n Puducherry , Puducherry...","Varun Dev \n Jalandhar , Punjab \n PERSONAL am...","Mahesh Chavan \n software developer \n Pune , ...",Rajeev Gupta \n additional Information \n * se...,"Surya V \n - end developer \n Chennai , Tamil ...","Deepali Chaudhari \n Mumbai , Maharashtra \n","N N Sriteja \n Hyderabad , ANDHRA PRADESH , 50...","Shirish nakhate \n SQL developer \n Pune , Mah...","Dilip Sambare \n Pandhurna , Madhya Pradesh \n...",...,Gandhi Gautam \n Software Developer - APYL sof...,"Jemini Lad \n Sachin , Gujarat \n additional I...",R Venkata Sai Sruthi \n year experience Hybrid...,Dinesh Maharajan \n Java UI Developer - Java U...,Neeraj Kumar \n trainee - NKB Software Solutio...,"Aniket Dhanrao \n Nashik , Maharashtra \n","Nilkanth Jagtap \n Pune , Maharashtra \n",satish t \n Application Developer - United Hea...,Kirubakaran Srinivasan \n End UI Developer Ang...,RESUME \n SOHAM DAS \n currently pursue m.tech...
